# Posterior of token-level parameters

This notebook computes the sufficient statistics for token-level parameters from (partially) matched name lists. Since all token parameters are independent given matches and global prior parameters, this can be done for each token individually.

The likelihood can be computed directly from the Beta-Binomial pmf,

$$
p(k \mid n, \alpha, \beta) = \frac{Z(\alpha + k, \beta + n - k)}{Z(\alpha, \beta)}
$$

where

$$
Z(\alpha, \beta) = \frac{\Gamma(\alpha)\Gamma(\beta)}{\Gamma(\alpha + \beta)}
$$

is the normalizer of the Beta-distribution.

We use a mean-strength parametrization

$$
\begin{align}
\alpha &= s \pi \\
\beta &= s (1 - \pi)
\end{align}
$$

$\pi \in [0, 1]$ is an the prior probability of an unseen entity to contain the token in its name, so that it uses entities as the denominator (not names). Consequently, the estimate changes when names are matched to entities. It is also weakly correlated with $s$. This tends to be relatively well determined by the data except for very rare tokens.

$s > 0$ measures how consistently a token is used: For $s \rightarrow 0$ we expect a token to occur with certainty in all names of an entity if it has occured in one name. For $s \rightarrow \infty$ the frequency is the population frequency regardless of whether the token was part of a name of an entity or not. Hence, low values of $s$ correspond to more relevant tokens, while tokens with large values are effectively ignored.

We impose a log-normal prior on $s$ and a flat prior on $\log\pi$, since $\pi$ is much better determined by the data, while for $s$ the likelihood is not even a normalizable distribution in some cases.


In principle, the likelihood contains one factor per entity, but in practice there are large multiplicities. For pairwise matches, the only possible values of the sufficient statistics are $n = 1, k = 0,1$ and $n=2, k=0,1,2$. We compute the likelihood for the distinct sufficient statistics only once while taking multiplicities into account.

In [ ]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

from ptfidf import utils as ut
from ptfidf.train.aggregation import get_group_statistics, compress_group_statistics
from ptfidf.train.likelihood import beta_binomial_log_likelihood
from ptfidf.train.inference import map_estimate

In [ ]:
def label_training_data(names, target, matched_symbols):
    """
    Label training data.
    
    Names with symbol in matched_symbols will be assigned to 
    the same entity (per symbol).
    
    Parameters
    ----------
    names : pandas.DataFrame
        columns ['source', 'symbol', ...]
    target : str
        value of source that is to be considered target list
    matched_symbols : sequence or int
        if sequence, these symbols are matched. If int,
        draw matched_symbols symbols randomly.
    Returns
    -------
    pandas.Series
        values are '{source}:{symbol}' where source equals target for
        matched symbols.
    """
    if isinstance(matched_symbols, int):
        matched_symbols = np.random.choice(np.unique(names['symbol']), replace=False, size=matched_symbols)
    res = pd.Series(index=names.index, name='entity')
    idx = names['symbol'].isin(matched_symbols)
    res.loc[idx] = target + ':' + names.loc[idx, 'symbol']
    res.loc[~idx] = names.loc[~idx, 'source'] + ':' + names.loc[~idx, 'symbol']
    return res

# Preprocessing

## Read name files and vectorize

In [ ]:
# read files and concatenate into single table
datadir = Path('../data').resolve()
filenames = ['wikipedia.csv', 'slickcharts.csv']
names = pd.concat([
    pd.read_csv(datadir.joinpath(fn)).assign(source=fn.split('.')[0]) for fn in filenames
], axis=0).sort_values(['symbol', 'source']).reset_index(drop=True)

# drop some symbols that are impossible to get correct with current preprocessing

# These differ only in appended Class <X>, where <X> is a single letter that is 
# dropped in the preprocessing
duplicate_companies = ['GOOGL', 'NWSA', 'DISCA', 'FOXA', 'UAA']
# No (or almost no) token overlap due to spelling variations
impossible_symbols = ['VFC', 'PHM', 'UNH', 'LLL']
names = names[~names['symbol'].isin(duplicate_companies + impossible_symbols)]

# vectorize
vec = CountVectorizer(binary=True).fit(names['name'])
X = vec.transform(names['name'])
vocabulary_inv = {v: k for k, v in vec.vocabulary_.items()}

## Label training data

Since there are many tokens with the same statistics (e.g. tokens that occur only once or twice), also compute distinct token statistics for more convenient display of interesting examples.

In [ ]:
labels = label_training_data(names, 'wikipedia', 400)
y = LabelEncoder().fit_transform(labels.values)
counts, nobs = get_group_statistics(X, y)
n, k, weights = compress_group_statistics(counts, nobs)

# get distinct token statistics and associated tokens
distinct_weights, distinct_idx = np.unique(weights, axis=0, return_inverse=True)

# A look at the posterior

Show joint distribution of token-level parameters for selected token

In [ ]:
# parameters
prior_mean = np.log(.2)
prior_std = 2.
max_token_display = 5

# grid
pivals = np.linspace(np.log(1e-5), np.log(.5), 100)
svals = np.linspace(np.log(1e-4), np.log(1e3), 100)
s, pi = np.meshgrid(svals, pivals)
alpha, beta = np.exp(s) * np.exp(pi), np.exp(s) * (1. - np.exp(pi))

In [ ]:
i = np.random.choice(distinct_weights.shape[0])

selected_tokens = np.where(distinct_idx == i)[0]

output = ', '.join(vocabulary_inv[t] for t in selected_tokens[:max_token_display])
if len(selected_tokens) > max_token_display:
    output += ', ... ({} more)'.format(len(selected_tokens) - max_token_display)
print('tokens:\n' + output)


post = beta_binomial_log_likelihood(alpha[..., None], beta[..., None], k, n).dot(distinct_weights[i])
post -= .5 * (s - prior_mean)**2 / prior_std**2
post = np.exp(post - post.max())
post /= post.sum()

pi_opt, s_opt = map_estimate(n, k, distinct_weights[i:i+1], prior_mean, prior_std)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))


lg10 = np.log(10)
ax = axes[0]
ax.set_xlabel('$log_{10} s$')
ax.set_ylabel('$log_{10} \pi$')
ax.pcolormesh(s / lg10, pi / lg10, post, cmap=plt.cm.Blues)
ax.plot([np.log10(s_opt)], [np.log10(pi_opt)], 'ro')
ax.grid()

ax = axes[1]
ax.set_title('marginal distribution $\log_{10}s$')
ax.plot(svals / lg10, post.sum(axis=0))
ax.axvline(np.log10(s_opt), color='r')

ax = axes[2]
ax.set_title('marginal distribution $\log_{10}\pi$')
ax.plot(pivals / lg10, post.sum(axis=1))
ax.axvline(np.log10(pi_opt), color='r')

pd.DataFrame({'n': n, 'k': k, 'weights': distinct_weights[i]}).set_index(['n', 'k'])['weights']